In [ ]:
import ipywidgets as widgets
import pandas as pd

In [ ]:
class Book():
    def __init__(self, path):
        self.pages = []
        df = pd.read_csv(path)
        
        page = []
        page_height = 0
        
        for _, row in df.iterrows():
            if page_height > 20:
                self.pages.append(page)
                page = []
                page_height = 0
            
            page_height += 1 if row.word == "\n" else len(row.word) / 100
            page.append(row)
        
        self.pages.append(page)
        
    def print_page(self, page_num, percentile):
        result = []

        for word in self.pages[page_num - 1]:
            if int(word.percentile) <= percentile:
                result.append("\033[96m" + word.word + "\033[0m")
            else:
                result.append(word.translation)

        print("".join(result))

In [ ]:
book = Book("fr/out/le_petit_prince.csv")

def back_widget_fn(b):
    page_widget.value = max(page_widget.value - 1, 1)

def next_widget_fn(b):
    page_widget.value = min(page_widget.value + 1, len(book.pages))

# To go backwards and forwards on pages.
back_widget = widgets.Button(description='Back', icon='arrow-left')
next_widget = widgets.Button(description='Next', icon='arrow-right')
back_widget.on_click(back_widget_fn)
next_widget.on_click(next_widget_fn)

# To display the current page and allow page teleportation.
page_widget = widgets.BoundedIntText(description='Page:', value=1, min=1, max=len(book.pages))
page_widget.layout = widgets.Layout(width='200px')

# "0" is entirely english, and "100" is entirely the foreign language.
slider = widgets.IntSlider(value=0, min=0, max=100, description='Difficulty:')

# Finally, the display.
out = widgets.interactive_output(book.print_page, {'page_num': page_widget, 'percentile': slider})
widgets.VBox([out, widgets.HBox([back_widget, next_widget, page_widget, slider])])